In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, models

from collections import Counter


## Loading in Dataset & Filtering it


In [ ]:
# optionally, here are functions save to csv file and reload it correctly from the csv

def load_boards_csv(filename):
    boards = []
    moves = []
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        rows = list(reader)

    i = 0
    while i < len(rows):
        # 1) Grab header row: [ "BOARD_INDEX", idx, "MOVE", move_val ]
        header = rows[i]
        # parse board index if needed: idx = int(header[1])
        move_val = int(header[3])
        moves.append(move_val)
        i += 1

        # 2) Grab 6 lines for the board
        board_rows = rows[i : i + 6]
        i += 6

        # Convert from shape (6 lines × 14 columns) => (6, 7, 2)
        board_6x7x2 = np.zeros((6, 7, 2), dtype=int)
        for row_idx in range(6):
            row_data = board_rows[row_idx]       # 14 entries
            for col_idx in range(7):
                ch0 = int(row_data[col_idx*2])
                ch1 = int(row_data[col_idx*2 + 1])
                board_6x7x2[row_idx, col_idx, 0] = ch0
                board_6x7x2[row_idx, col_idx, 1] = ch1

        boards.append(board_6x7x2)

        # 3) Skip blank line if it exists
        if i < len(rows) and len(rows[i]) == 0:
            i += 1

    boards = np.stack(boards, axis=0)  # shape (N, 6, 7, 2)
    moves = np.array(moves)
    return boards, moves

import csv
import numpy as np

def load_boards_csv(filename):
    """
    Reads a CSV file where each board is stored as:
      1) A header row: ["BOARD_INDEX", i, "MOVE", moves[i]]
      2) 6 rows, each with 14 integers (7 columns × 2 channels)
      3) A blank line (optional) separating each board

    Returns:
      boards: ndarray of shape (N, 6, 7, 2)
      moves: ndarray of shape (N,)
    """
    boards = []
    moves = []

    with open(filename, 'r', newline='') as f:
        reader = csv.reader(f)
        rows = list(reader)

    i = 0
    while i < len(rows):
        # 1) Read the header row (e.g. ["BOARD_INDEX", "0", "MOVE", "3"])
        header = rows[i]
        move_val = int(header[3])   # parse the move
        moves.append(move_val)
        i += 1

        # 2) Read the 6 lines of board data
        board_rows = rows[i : i + 6]
        i += 6

        # Convert from shape (6, 14) -> (6, 7, 2)
        board_6x7x2 = np.zeros((6, 7, 2), dtype=int)
        for row_idx in range(6):
            row_data = board_rows[row_idx]  # 14 entries
            for col_idx in range(7):
                ch0 = int(row_data[col_idx * 2])
                ch1 = int(row_data[col_idx * 2 + 1])
                board_6x7x2[row_idx, col_idx, 0] = ch0
                board_6x7x2[row_idx, col_idx, 1] = ch1

        boards.append(board_6x7x2)

        # 3) Skip the blank line if present
        if i < len(rows) and len(rows[i]) == 0:
            i += 1

    boards = np.stack(boards, axis=0)  # shape (N, 6, 7, 2)
    moves = np.array(moves)
    return boards, moves


In [ ]:
boards, moves = load_boards_csv("/kaggle/input/dan-connect4-data/dan_connect4_data.csv")
print(boards.shape)  # e.g. (N, 6, 7, 2)
print(moves.shape)   # e.g. (N,)

## CG Data Pre-processing

In [ ]:

import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(boards, dtype=np.float32)  

print(f"X shape: {X.shape}")  

y = np.array(moves, dtype=np.int32)  

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=15)

print(f"Training Set Shape: {X_train.shape}, {y_train.shape}")
print(f"Validation Set Shape: {X_val.shape}, {y_val.shape}")


## Finalized CNN 68.75%

In [ ]:
from tensorflow.keras import layers, models, regularizers, callbacks, optimizers

def no_max_cnn():
    model = models.Sequential()

    model.add(layers.Input(shape=(6, 7, 2)))

    # First Convolutional Layer
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Dropout(0.2))  

    # Second Convolutional Layer
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Dropout(0.3))  

    # Third Convolutional Layer
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Dropout(0.4))  

    # Fourth Convolutional Layer
    model.add(layers.Conv2D(1024, (3, 3), activation='relu', padding='same'))
    model.add(layers.Dropout(0.5))  

    model.add(layers.GlobalAveragePooling2D())  

    model.add(layers.Dense(2048, activation='relu', 
                           kernel_regularizer=regularizers.l2(0.01))) 

    model.add(layers.Dense(7, activation='softmax'))

    optimizer = optimizers.Adam(learning_rate=0.0006)  
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [ ]:
# Early Stopping Callback
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy',  
    patience=4,              
    min_delta=0.001,          
    restore_best_weights=True  
)

# Learning Rate Scheduler Callback
lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.1,             
    patience= 4,             
    min_lr=1e-6              
)

In [ ]:
m_model = no_max_cnn()

history = m_model.fit(
    X_train, y_train,
    epochs=50,  
    validation_data=(X_val, y_val),
    batch_size=64,
    callbacks=[lr_scheduler, early_stopping]

)


In [ ]:
m_model.count_params()

In [ ]:
print(epochs)
print(train_accuracies)
print(val_accuracies)


### Save the Model

In [ ]:
# After training the model
m_model.save('cnn_model.h5')


### Plot the Accuracies

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Extract accuracy and loss values
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plotting Accuracy
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training Accuracy')
    plt.plot(epochs, val_acc, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotting Loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Call the plotting function after training
plot_training_history(history)
